<a href="https://colab.research.google.com/github/chinmay-sh/StockAnalysisAI/blob/main/TickerAnalysisAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gradio -q
!pip install yfinance -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.8 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import yfinance as yf
import gradio as gr
import google.generativeai as genai
from google.colab import userdata
import requests
import math
import plotly.express as px
import json
import pandas as pd

In [3]:
ALPHAVANTAGE_API_KEY = userdata.get('ALPHAVANTAGE_API_KEY')

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')


# ALPHAVANTAGE_API_KEY = 'demo'

In [4]:
# prompt: generate a function for calling alphavantage api, sample api call for news sentiment: https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=demo, also limit to top 10 latest articles and only care about sentiment for the provided ticker

def get_news_sentiment(ticker):
    """
    Retrieves news sentiment for a given ticker from the Alpha Vantage API.

    Args:
        ticker: The stock ticker symbol (e.g., "AAPL").
        apikey: Your Alpha Vantage API key.

    Returns:
        A list of dictionaries, where each dictionary represents a news article
        and contains the article's title, summary, and sentiment score for the
        specified ticker.  Returns an empty list if there's an error or no data.
    """

    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={ALPHAVANTAGE_API_KEY}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        data = response.json()

        if "feed" not in data or not data["feed"]:
            print(f"No news sentiment data found for this ticker: {data['Information']}")
            return [], None

        # Limit to the top 10 latest articles and filter by ticker sentiment
        articles = []
        average_sentiment_score = 0
        total_found_tickers = 0
        for item in data["feed"][:10]:
            for sentiment in item["ticker_sentiment"]:
                if sentiment["ticker"] == ticker:
                    articles.append({
                        "title": item["title"],
                        "summary": item["summary"],
                        "url": item["url"],
                        "sentiment_score": sentiment["ticker_sentiment_score"],
                        "sentiment_label": sentiment["ticker_sentiment_label"]
                    })
                    average_sentiment_score += float(sentiment["ticker_sentiment_score"])
                    total_found_tickers += 1
                    break  # Stop once we find the sentiment for the ticker

        if total_found_tickers > 0:
            average_sentiment_score /= total_found_tickers
        return articles, average_sentiment_score

    except requests.exceptions.RequestException as e:
        print(f"Error fetching news sentiment data: {e}")
        return [], None
    except KeyError as e:
        print(f"Error parsing API response: Missing key {e}")
        return [], None


In [5]:
import ast

def llm_output_parser(response_text):
    """Parses the response text and returns a dictionary.

    Args:
        response_text: The response text which looks like a dict as a string.

    Returns:
        A dictionary containing the parsed data, or None if parsing fails.
    """
    try:
        response_data = response_text.split("```json")[1].split("```")[0]
        response_data = ast.literal_eval(response_data)
        return response_data
    except Exception as e:
        print("Error parsing LLM Output:", e)
        return None

In [9]:
# prompt: function to get today's date out as string

from datetime import date

def get_todays_date_as_string():
  """
  Returns today's date as a string in YYYY-MM-DD format.
  """
  today = date.today()
  return today.strftime("%Y-%m-%d")

In [13]:
def predict_stock_price(ticker):
    ticker = ticker.upper()
    # Get historical data
    stock_data = yf.Ticker(ticker)
    hist = stock_data.history(period="5d")


    hist_closing_prices = {}
    for key, value in hist["Close"].to_dict().items():
        hist_closing_prices[str(key).split(" ")[0]] = math.floor(value)

    # Get current price
    current_price = stock_data.info['currentPrice']

    # Get news sentiment from Alpha Vantage
    news_sentiment_data_alphavantage, average_sentiment_score_alphavantage = get_news_sentiment(ticker)

    # Prepare prompt for LLM
    system_instructions = """
    You are a stock/cryptocurrency price and news sentiment analysis model. Special Note: Stock markets are closed on the weekends, so no prediction for weekend dates.

    Input will be stock/cryptocurrency ticker along with current price, 5 days of historical daily price, news sentiment average from AlphaVantage API for latest 10 news articles, and 10 latest news articles will also be input.

    Your task is to output a summary of those 10 news articles along with estimates of closing price for next 3 days. This estimate should be based on price trends along with news sentiment analysis and knowledge gained from news articles.

    Inputs:
    ===============
    ticker: [string]
    date_today: [string]
    current_price: [float]
    historical_price_data: [dict of historical closing prices for last 5 days week]
    news_sentiment_average_alphavantage: [Average of news sentiment scores for latest 10 news articles from AlphaVantage API (x, where x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish)]
    news_articles_list: [list of dict data of Latest 10 news articles related to ticker from AlphaVantage API]


    Outputs (in JSON format):
    ===============
    predicted_prices: Dict[str,float] (python dict of predicted closing prices for next 3 days)
    summary_of_news_articles: [list of summary items for each of 10 articles, each list item is a dict which should be like this: {'title': 'article_original_title','summary_llm_based': 'text ...','url': 'original_url'}, summary text should be in 100 words, also escaping any special character which might result in errors while parsing]
    collective_summary: [summary of all 10 news articles in 300 words]
    explanation: [explanation of how the model arrived at the predicted prices and collective summary, in 300 words, also escaping any special character which might result in errors while parsing]

    Example for prompts:
    ===============
    Input:
    ------
    ticker: "AAPL"
    date_today: "2024-10-02"
    current_price: 150.0
    historical_price_data: {'2024-10-28': 426,'2024-10-29': 431,'2024-10-30': 432,'2024-10-31': 406,'2024-11-01': 410}
    news_sentiment_average_alphavantage: 0.2
    news_articles_list: [{'title': 'Why Qorvo Stock Tanked This Week',
        'summary': 'The chip supplier really had a fiscal second quarter to forget.',
        'url': 'https://www.fool.com/investing/2024/11/01/why-qorvo-stock-tanked-this-week/',
        'sentiment_score': '-0.036857',
        'sentiment_label': 'Neutral'},
      {'title': "If You'd Invested $1,000 in Apple Stock 10 Years Ago, Here's How Much You'd Have Today",
        'summary': 'Apple has delivered incredible returns for long-term investors.',
        'url': 'https://www.fool.com/investing/2024/11/01/if-youd-invested-1000-in-apple-stock-10-years-ago/',
        'sentiment_score': '0.483551',
        'sentiment_label': 'Bullish'},
        .....
      {'title': "Apple Q4 Results 'Better-Than-Feared': Analysts Highlight iPhone 16 Demand, Apple Intelligence Opportunity After 'Rock Solid Quarter' - Apple  ( NASDAQ:AAPL ) ",
        'summary': "Apple Inc AAPL beat analysts' fourth-quarter revenue and earnings estimates, but guidance that came in below estimates has analysts questioning the future outlook after Thursday's report. Bank of America analyst Wamsi Mohan reiterated a Buy rating on Apple with a $256 price target.",
        'url': 'https://www.benzinga.com/analyst-ratings/analyst-color/24/11/41690576/apple-q4-results-better-than-feared-analysts-highlight-iphone-16-demand-apple-intel',
        'sentiment_score': '0.34258',
        'sentiment_label': 'Somewhat-Bullish'}]

    Output (in JSON format):
    ------
    {
    "future_prices_3_days": {'2024-11-02': price, '2024-11-03': price, '2024-11-04': price}
    "summary_of_news_articles": [{'title': 'article1_original_title','summary_llm_based': 'text ...','url': 'original_url1'}, ..., {'title': 'article10_original_title','summary_llm_based': 'text ...','url': 'original_url10'}]
    "collective_summary": "summary of all 10 news articles"
    "explanation_of_prediction": "explanation of how the model arrived at the future prices and the collective summary"
    }
    """
    model = 'gemini-1.5-flash' # @param {type: "string"} ["gemini-1.5-pro", "gemini-1.5-flash"]
    temperature = 0.5 # @param {type: "slider", min: 0, max: 2, step: 0.05}

    genai.configure(api_key=GEMINI_API_KEY)

    model = genai.GenerativeModel(model, system_instruction=system_instructions)
    config = genai.GenerationConfig(temperature=temperature)

    prompt = f"""
    Input:
    ------
    ticker: {ticker}
    date_today: {get_todays_date_as_string()}
    current_price: {stock_data.info['currentPrice']}
    historical_price_data: {hist_closing_prices}
    news_sentiment_average_alphavantage: {average_sentiment_score_alphavantage}
    news_articles_list: {news_sentiment_data_alphavantage}

    Outputs (in JSON format):
    ===============
    predicted_prices: [dict of predicted closing prices for next 3 days]
    summary_of_news_articles: [list of summary items of each of latest 10 news articles]
    collective_summary: [summary of all 10 news articles]
    explanation_of_prediction: [explanation of how the model arrived at the future prices and the collective summary]
    """

    response = model.generate_content(contents=[prompt], generation_config=config)

    # return (response.text)

    return response.text, ticker, stock_data.info['currentPrice'], hist_closing_prices, news_sentiment_data_alphavantage



In [14]:
def plot_maker_prices(df, x_item, y_item):
    plot = px.line(df,x=x_item,y=y_item, text=y_item)
    plot.update_traces(textposition="bottom right")
    return plot

In [15]:
def result_parser(ticker):
    output_text, ticker, currentPrice, hist_closing_prices, news_sentiment_data_alphavantage= predict_stock_price(ticker)

    outputDict = llm_output_parser(output_text)

    hist_closing_prices_df = pd.DataFrame(list(hist_closing_prices.items()), columns=['Date', 'Stock Closing Price'])

    predicted_prices = pd.DataFrame(list(outputDict["predicted_prices"].items()), columns=['Date', 'Predicted Closing Price'])
    summary_of_news_articles_llm = pd.DataFrame(list(outputDict["summary_of_news_articles"]))
    collective_summary = str(outputDict["collective_summary"])
    explanation_of_prediction = str(outputDict["explanation_of_prediction"])

    news_sentiment_data_alphavantage = pd.DataFrame(news_sentiment_data_alphavantage)

    plot_historical = plot_maker_prices(hist_closing_prices_df,'Date','Stock Closing Price')

    plot_preidcted_prices = plot_maker_prices(predicted_prices,'Date','Predicted Closing Price')

    return predicted_prices, ticker, currentPrice, hist_closing_prices_df, news_sentiment_data_alphavantage, summary_of_news_articles_llm, collective_summary, explanation_of_prediction, plot_historical, plot_preidcted_prices

with gr.Blocks() as app:
    gr.Markdown("# Stock Ticker Analysis & Prediction App")

    ticker_input = gr.Textbox(
        label="Enter Ticker for analysis (support only for NASDAQ tickers / Yahoo Finance)",
        placeholder="e.g., aapl/msft"
    )

    analyze_button = gr.Button("Analyze", variant='primary')


    gr.Markdown("## Stock Details")
    with gr.Row():
        ticker_output = gr.Textbox(label="Ticker")
        current_price_output = gr.Textbox(label="Current Price")

    with gr.Row():
        historical_price_data_output = gr.Dataframe(label="Historical Price Data",headers=["Date", "Stock Closing Price"],datatype=["str", "number"])
        plot_historical_output = gr.Plot(label="Plot for historical prices")

    news_sentiment_alphavantage_output = gr.Dataframe(label="News Sentiment Results AlphaVantage API", headers=["Title", "Summary", "URL", "Sentiment Score", "Sentiment Label"])

    gr.Markdown("## Stock Price Prediction")

    with gr.Row():
        prediction_output = gr.Dataframe(label="LLM Predicted Closing Prices", headers=["Date", "Predicted Closing Price"], datatype=["str", "number"])
        plot_predicted_prices_output = gr.Plot(label="Plot for predicted prices")

    gr.Markdown("## Summary of News Articles")
    summary_of_news_articles_output = gr.Dataframe(label="Summary of Latest 10 News Articles from LLM", headers=["Title", "LLM Summary", "URL"])
    with gr.Row():
        collective_summary_output = gr.Textbox(label="Summary of all 10 news articles")
        explanation_of_prediction_output = gr.Textbox(label="Explanation of how the model arrived at the future prices and the collective summary")




    analyze_button.click(
        result_parser,
        inputs=[ticker_input],
        outputs=[prediction_output,
                 ticker_output,
                 current_price_output,
                 historical_price_data_output,
                 news_sentiment_alphavantage_output,
                 summary_of_news_articles_output,
                 collective_summary_output,
                 explanation_of_prediction_output,
                 plot_historical_output,
                 plot_predicted_prices_output
                 ],
    )

# Launch the app
app.launch(share=True, debug=True, show_error=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://55ccf5b87c9c487040.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://55ccf5b87c9c487040.gradio.live
